In [ ]:
#Setup and Dataset Downloading

# Install necessary packages
!pip install kagglehub tensorflow pillow pandas matplotlib

# Import KaggleHub to download the dataset
import kagglehub

# Download the CelebA dataset
path = kagglehub.dataset_download("jessicali9530/celeba-dataset")
print("Path to dataset files:", path)


100%|██████████| 1.33G/1.33G [00:18<00:00, 75.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/jessicali9530/celeba-dataset/versions/2


In [ ]:
# Dataset Path and Preprocessing

import os
import pandas as pd
import numpy as np
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt

# Paths to dataset files (assuming default paths after KaggleHub download)
data_dir = os.path.join(path, 'img_align_celeba/img_align_celeba')
attr_path = os.path.join(path, 'list_attr_celeba.csv')
partition_path = os.path.join(path, 'list_eval_partition.csv')

img_size = 128

# Load attributes and partition files
attributes = pd.read_csv(attr_path)
partition = pd.read_csv(partition_path)

# Preprocess images (filter for smiling faces)
smiling_images = attributes[attributes['Smiling'] == 1]['image_id']
image_paths = smiling_images.map(lambda x: os.path.join(data_dir, x))

def preprocess_image(image_path):
    img = Image.open(image_path).resize((img_size, img_size))
    img = np.asarray(img) / 255.0  # Normalize to [0, 1]
    return img

images = np.array([preprocess_image(img) for img in image_paths[:10000]])
print(f"Loaded {len(images)} smiling images.")


Loaded 10000 smiling images.


In [ ]:
#Building the Generator

from tensorflow.keras.layers import Dense, Reshape, Conv2DTranspose, LeakyReLU, BatchNormalization

def build_generator():
    model = tf.keras.Sequential([
        Dense(8 * 8 * 256, input_dim=100, activation='relu'),
        Reshape((8, 8, 256)),
        BatchNormalization(),
        LeakyReLU(0.2),

        Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),

        Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),

        Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', activation='tanh')
    ])
    return model
   model.summary()


In [ ]:
#Building the Discriminator

from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout

def build_discriminator():
    model = tf.keras.Sequential([
        Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=(64, 64, 3)),
        LeakyReLU(0.2),
        Dropout(0.3),

        Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        LeakyReLU(0.2),
        Dropout(0.3),

        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    return model
    model.summary()



In [ ]:
# Preprocessing the Dataset Correctly

img_size = 64  # Ensure generator and discriminator work with 64x64

def preprocess_image(image_path):
    img = Image.open(image_path).resize((img_size, img_size))  # Resize to 64x64
    img = np.asarray(img) / 127.5 - 1.0  # Normalize to [-1, 1] for tanh activation
    return img

images = np.array([preprocess_image(img) for img in image_paths[:10000]])



In [ ]:
# Training the GAN-Loss Functions and Optimizers

cross_entropy = tf.keras.losses.BinaryCrossentropy()

generator = build_generator()
discriminator = build_discriminator()

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Training Loop

@tf.function
def train_step(real_images):
    noise = tf.random.normal([BATCH_SIZE, 100])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        real_output = discriminator(real_images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_gen = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_disc = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_gen, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_disc, discriminator.trainable_variables))

    return gen_loss, disc_loss

EPOCHS = 300
BATCH_SIZE = 16

for epoch in range(EPOCHS):
    gen_loss_epoch, disc_loss_epoch = [], []
    for i in range(0, len(images), BATCH_SIZE):
        batch = images[i:i + BATCH_SIZE]
        if len(batch) == BATCH_SIZE:
            gen_loss, disc_loss = train_step(batch)
            gen_loss_epoch.append(gen_loss.numpy())
            disc_loss_epoch.append(disc_loss.numpy())

    print(f'Epoch {epoch+1}/{EPOCHS} | Gen Loss: {np.mean(gen_loss_epoch):.4f} | Disc Loss: {np.mean(disc_loss_epoch):.4f}')

Epoch 1/300 | Gen Loss: 2.7100 | Disc Loss: 0.4337
Epoch 2/300 | Gen Loss: 1.7575 | Disc Loss: 0.8610
Epoch 3/300 | Gen Loss: 1.4510 | Disc Loss: 0.8548
Epoch 4/300 | Gen Loss: 1.5672 | Disc Loss: 0.9061
Epoch 5/300 | Gen Loss: 1.4112 | Disc Loss: 0.8851
Epoch 6/300 | Gen Loss: 1.4192 | Disc Loss: 0.8933
Epoch 7/300 | Gen Loss: 1.3590 | Disc Loss: 0.9062
Epoch 8/300 | Gen Loss: 1.4142 | Disc Loss: 0.8984
Epoch 9/300 | Gen Loss: 1.4583 | Disc Loss: 0.8275
Epoch 10/300 | Gen Loss: 1.5295 | Disc Loss: 0.7968
Epoch 11/300 | Gen Loss: 1.5454 | Disc Loss: 0.8283
Epoch 12/300 | Gen Loss: 1.5062 | Disc Loss: 0.8685
Epoch 13/300 | Gen Loss: 1.5778 | Disc Loss: 0.8351
Epoch 14/300 | Gen Loss: 1.5992 | Disc Loss: 0.8464
Epoch 15/300 | Gen Loss: 1.5497 | Disc Loss: 0.8700
Epoch 16/300 | Gen Loss: 1.5732 | Disc Loss: 0.8366
Epoch 17/300 | Gen Loss: 1.6722 | Disc Loss: 0.7851
Epoch 18/300 | Gen Loss: 1.6911 | Disc Loss: 0.7923
Epoch 19/300 | Gen Loss: 1.8205 | Disc Loss: 0.7311
Epoch 20/300 | Gen Lo

In [ ]:
# prompt: By analyzing the previous code statement, I want to save and download the model I am training

# Save the generator model
generator.save('/content/drive/My Drive/generator_model1')

# Save the discriminator model
discriminator.save('/content/drive/My Drive/discriminator_model1')

print("Models saved to Google Drive.")

#To download the model:
#from google.colab import files
#files.download('/content/drive/My Drive/generator_model')
#files.download('/content/drive/My Drive/discriminator_model')

In [ ]:
import IPython.display as display
from google.colab import files
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Function to process the uploaded image
def preprocess_uploaded_image(image_path):
    img = Image.open(image_path).convert("RGB")  # Convert to RGB to handle all formats
    img = img.resize((64, 64))  # Resize to match the model input
    img_array = np.asarray(img) / 127.5 - 1.0  # Normalize to [-1, 1]
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Function to generate a smiling face
def generate_smiling_face(generator):
    print("Please upload an image (any format like .jpg, .png, etc.)")
    uploaded = files.upload()  # Upload the image
    for image_name in uploaded.keys():
        print(f"Image {image_name} uploaded successfully!")

        # Preprocess the uploaded image
        input_img = preprocess_uploaded_image(image_name)

        # Generate the smiling face using the generator
        noise = tf.random.normal([1, 100])
        generated_image = generator(noise, training=False)

        # Convert the generated image from [-1, 1] to [0, 1] range for display
        generated_image = (generated_image[0] + 1) / 2.0

        # Display the original and generated images side by side
        plt.figure(figsize=(8, 4))
        plt.subplot(1, 2, 1)
        plt.title("Uploaded Image")
        plt.imshow(np.asarray(Image.open(image_name).resize((64, 64))))
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.title("Smiling Face")
        plt.imshow(generated_image.numpy())
        plt.axis('off')

        plt.show()

# Run the function to upload and generate a smiling face
generate_smiling_face(generator)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')